In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('sign_mnist_train.csv')

In [3]:
test = pd.read_csv('sign_mnist_test.csv')

In [4]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [5]:
train.shape

(27455, 785)

In [8]:
labels = train['label'].values

In [9]:
train.drop('label', axis = 1,inplace=True)

In [11]:
labels.shape

(27455,)

In [12]:
from sklearn.preprocessing import LabelBinarizer
label_binrizer = LabelBinarizer()
labels = label_binrizer.fit_transform(labels)

In [13]:
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [14]:
labels.shape

(27455, 24)

In [26]:
images = train.values
images = np.array([np.reshape(i, (28, 28)) for i in images])


In [27]:
images.shape

(27455, 28, 28)

In [40]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.3, random_state = 101)

In [31]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

Using TensorFlow backend.


In [32]:
batch_size = 128
num_classes = 24
epochs = 50

In [41]:
x_train = x_train /255 
x_train.shape

(19218, 28, 28)

In [42]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

In [43]:
x_train.shape

(19218, 28, 28, 1)

In [44]:
x_test = x_test / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [45]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28, 28 ,1) ))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.20))
model.add(Dense(num_classes, activation = 'softmax'))

In [46]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [47]:
history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=epochs, batch_size=batch_size)

Train on 19218 samples, validate on 8237 samples
Epoch 1/50
19218/19218 [==============================] - 24s 1ms/step - loss: 2.5045 - accuracy: 0.2327 - val_loss: 1.4188 - val_accuracy: 0.5797
Epoch 2/50
19218/19218 [==============================] - 21s 1ms/step - loss: 1.1142 - accuracy: 0.6281 - val_loss: 0.6873 - val_accuracy: 0.7982
Epoch 3/50
19218/19218 [==============================] - 19s 1ms/step - loss: 0.6636 - accuracy: 0.7777 - val_loss: 0.4158 - val_accuracy: 0.8711
Epoch 4/50
19218/19218 [==============================] - 20s 1ms/step - loss: 0.4212 - accuracy: 0.8588 - val_loss: 0.2433 - val_accuracy: 0.9278
Epoch 5/50
19218/19218 [==============================] - 19s 1ms/step - loss: 0.2805 - accuracy: 0.9101 - val_loss: 0.1599 - val_accuracy: 0.9576
Epoch 6/50
19218/19218 [==============================] - 20s 1ms/step - loss: 0.1926 - accuracy: 0.9392 - val_loss: 0.1091 - val_accuracy: 0.9738
Epoch 7/50
19218/19218 [==============================] - 20s 1ms/ste

In [57]:
model.save('CNN_1.h5')

In [54]:
test_labels = test['label']
test.drop('label', axis = 1, inplace = True)
test_images = test.values
test_images = np.array([np.reshape(i, (28, 28)) for i in test_images])
test_images = np.array([i.flatten() for i in test_images])
test_labels = label_binrizer.fit_transform(test_labels)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)
test_images.shape

(7172, 28, 28, 1)

In [55]:
y_pred = model.predict(test_images)

In [56]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, y_pred.round())

0.899191299498048